In [30]:
import pandas as pd

import pymongo
from dotenv import load_dotenv
import os
from pymongo import UpdateMany

load_dotenv()

mongodb_uri = os.environ.get("MONGODB_URI")

client = pymongo.MongoClient(mongodb_uri)
db = client[os.environ.get("MONGODB_DB")]
classes = db['classes']

In [31]:
df = pd.read_csv('./uiuc-gpa-dataset.csv')  
df.head()

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,B-,C+,C,C-,D+,D,D-,F,W,Primary Instructor
0,2023,Spring,2023-sp,AAS,100,Intro Asian American Studies,DIS,0,11,5,...,0,0,0,0,0,0,0,1,0,"Shin, Jeongsu"
1,2023,Spring,2023-sp,AAS,100,Intro Asian American Studies,DIS,0,17,2,...,1,0,0,0,0,0,0,0,1,"Shin, Jeongsu"
2,2023,Spring,2023-sp,AAS,100,Intro Asian American Studies,DIS,0,13,2,...,2,0,0,1,0,0,0,1,0,"Lee, Sabrina Y"
3,2023,Spring,2023-sp,AAS,200,U.S. Race and Empire,LCD,6,15,5,...,0,0,0,0,0,1,0,1,0,"Sawada, Emilia"
4,2023,Spring,2023-sp,AAS,215,US Citizenship Comparatively,LCD,16,12,2,...,1,0,0,0,0,0,0,0,0,"Kwon, Soo Ah"


In [32]:
# Group by course number and title, adding up the total number of students and total number of students who got an A
df_grouped = df.groupby(['Subject', 'Number']).agg({'A+': 'sum', 'A': 'sum', 'A-': 'sum', 'B+': 'sum', 'B': 'sum', 'B-': 'sum', 'C+': 'sum', 'C': 'sum', 'C-': 'sum', 'D+': 'sum', 'D': 'sum', 'D-': 'sum', 'F': 'sum', 'W': 'sum'}).reset_index()
df_grouped.head()

,Subject,Number,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W
0,AAS,100,805,2064,703,445,470,197,102,123,44,22,25,17,102,17
1,AAS,105,23,20,3,3,4,1,0,1,0,0,1,0,1,0
2,AAS,120,34,149,130,102,106,50,39,33,12,6,11,1,2,5
3,AAS,200,30,43,31,16,9,10,2,1,1,1,3,1,2,0
4,AAS,211,3,20,12,2,4,3,2,1,0,3,0,0,2,0


In [33]:
results = []
#Loop over each row in the dataframe
for index, row in df_grouped.iterrows():
    # Create a dictionary with the data we want to insert
    num_students = row['A+'] + row['A'] + row['A-'] + row['B+'] + row['B'] + row['B-'] + row['C+'] + row['C'] + row['C-'] + row['D+'] + row['D'] + row['D-'] + row['F'] + row['W']
    avg_gpa = (4*row['A+'] + 4*row['A'] + 3.67*row['A-'] + 3.33*row['B+'] + 3*row['B'] + 2.67*row['B-'] + 2.33*row['C+'] + 2*row['C'] + 1.67*row['C-'] + 1.33*row['D+'] + 1*row['D'] + 0.67*row['D-'])/num_students
    percent_4s = (row['A+'] + row['A'])/ num_students
    # Add the dictionary to the results list
    results.append(UpdateMany({'number' : str(row['Number']), 'code' : row['Subject']}, {'$set': {'grades' : {'avg_gpa': avg_gpa, 'percent_4s': percent_4s, 'num_students': num_students}}}))
    

In [34]:
db_res = classes.bulk_write(results)


In [35]:
print(results[0])

UpdateMany({'number': '100', 'code': 'AAS'}, {'$set': {'grades': {'avg_gpa': 3.5234890965732086, 'percent_4s': 0.5586059190031153, 'num_students': 5136}}}, False, None, None, None)


In [36]:
print(db_res.bulk_api_result)

{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 6271, 'nModified': 3139, 'nRemoved': 0, 'upserted': []}
